In [1]:
import numpy as np
import qutip
import pickle
import nevergrad as ng
from CoupledQuantumSystems.drive import DriveTerm, square_pulse_with_rise_fall
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process
from concurrent.futures import ProcessPoolExecutor
# -------------------------------
# Define your parameters and objects
# -------------------------------
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [
    qutip.basis(qbt.truncated_dim, i) * qutip.basis(qbt.truncated_dim, i).dag()
    for i in range(10)
]

element = np.abs(
    qbt.fluxonium.matrixelement_table("n_operator", evals_count=3)[1, 2]
)
freq = (qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]) * 2 * np.pi


def objective(t_tot, amp, w_d,ramp):
    tlist = np.linspace(0, t_tot * (1+ramp), 100)

    initial_states = [
        qutip.basis(qbt.truncated_dim, 1),
        qutip.basis(qbt.truncated_dim, 2),
    ]

    drive_terms = [
        DriveTerm(
            driven_op=qutip.Qobj(qbt.fluxonium.n_operator(energy_esys=True)),
            pulse_shape_func=square_pulse_with_rise_fall,
            pulse_id="pi",
            pulse_shape_args={
                "w_d": w_d,    # No extra 2pi factor
                "amp": amp,    # No extra 2pi factor
                "t_square": t_tot * (1-ramp),
                "t_rise": t_tot * ramp,
            },
        )
    ]

    # Solve the dynamics for each initial state
    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
            print_progress=False,
        )
        results.append(res)

    # Calculate final populations
    # (We want to transfer |1> -> |2> and |2> -> |1>.)
    one_minus_pop2 = abs(1 - results[0].expect[2][-1])  # from state |1> to |2>
    one_minus_pop1 = abs(1 - results[1].expect[1][-1])  # from state |2> to |1>

    return one_minus_pop2 + one_minus_pop1

# ---------------------------------------------------------------------
# SET UP NEVERGRAD INSTRUMENTATION
# ---------------------------------------------------------------------
# We'll specify a search space for each of t_tot, amp, w_d.
# For example, let's guess t_tot is in [1, 300].
# We'll assume amp in [0, 0.1], w_d in [freq*0.9, freq*1.1] as an example.
# Adjust these bounds to your scenario.


t_tot_list = np.linspace(50, 250, 21)
for t_tot in t_tot_list:
    # init_amp = np.pi / element / t_tot / 3.1415 / 2
    init_wd = qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
    amp_guess = 50/t_tot * 22
    parametrization = ng.p.Instrumentation(
        amp=ng.p.Scalar(init=amp_guess, lower=amp_guess/5, upper=amp_guess*5),        # or choose your own guess
        w_d=ng.p.Log(init=init_wd, lower=0.003, upper=0.02),
        ramp=ng.p.Scalar(init=0.15, lower=1e-10, upper=0.5),
    )
    # ---------------------------------------------------------------------
    # CHOOSE THE OPTIMIZER
    # ---------------------------------------------------------------------
    # Some popular choices: "CMA", "TwoPointsDE", "OnePlusOne", "PSO", "NGOpt", etc.
    # We'll pick CMA as an example. We'll also set a budget (# of function evaluations)
    # and a concurrency level (num_workers) for parallelizing the objective calls.
    budget = 1000
    num_workers = 20 
    optimizer = ng.optimizers.CMA(parametrization=parametrization,
                                budget=budget,
                                num_workers=num_workers)
    # ---------------------------------------------------------------------
    # RUN THE OPTIMIZATION
    # (Nevergrad automatically parallelizes if num_workers > 1.)
    # ---------------------------------------------------------------------
    def nevergrad_objective( amp, w_d,ramp):
        return objective(t_tot=t_tot, amp=amp, w_d=w_d,ramp=ramp)
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        recommendation = optimizer.minimize(nevergrad_objective,batch_mode=True,verbosity=2,executor=executor)

    # Extract best values
    best_amp = recommendation.kwargs["amp"]
    best_w_d = recommendation.kwargs["w_d"]
    best_ramp = recommendation.kwargs["ramp"]
    best_cost = recommendation.value

    print(f"Best cost found: {best_cost}")
    print(f"Best amp: {best_amp}, best w_d: {best_w_d}, best ramp: {best_ramp}")

    # Optionally, store results in a pickle
    results_dict = {
        "best_cost": best_cost,
        "best_amp": best_amp,
        "best_w_d": best_w_d,
        "best_ramp": best_ramp,
        "recommendation": recommendation,  # the full nevergrad result object
    }

    with open(f"nevergrad_optimized_results_{t_tot}.pkl", "wb") as f:
        pickle.dump(results_dict, f)


Updating fitness with value 0.0370713459142642
Updating fitness with value 1.693385637031012
Updating fitness with value 0.24494663266314975
Updating fitness with value 0.1632527556741581
Updating fitness with value 0.06937567213107732
Updating fitness with value 0.029364594057765703
Updating fitness with value 0.10248247382628017
Updating fitness with value 0.10971089361271369
Updating fitness with value 0.28512122549294516
Updating fitness with value 0.19593938716241577
Updating fitness with value 1.951726192139153
Updating fitness with value 1.157128909725747
Updating fitness with value 1.9333770466187656
Updating fitness with value 1.2967487054717626
Updating fitness with value 1.8614014361309394
Updating fitness with value 1.2145729413933948
Updating fitness with value 0.21270837673683085
Updating fitness with value 1.5997858428773504
Updating fitness with value 1.1259507985670094
Updating fitness with value 1.278372798121982
860 remaining budget and 0 running jobs
Current pessimi

 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 0.6180389737681365
Updating fitness with value 0.8868250276345006
Updating fitness with value 1.6215210196986805
Updating fitness with value 1.7925667078100929
Updating fitness with value 0.3093159840279128
Updating fitness with value 1.4067315392129922
Updating fitness with value 1.2815649953775925
Updating fitness with value 1.4044307758283212
Updating fitness with value 1.3993143522982625
Updating fitness with value 1.0379186575181445
Updating fitness with value 1.0220296905355948
Updating fitness with value 0.8374954002582248
Updating fitness with value 0.443192861257222
Updating fitness with value 0.9165741089487057
980 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 0.07568003298553183, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(3.1428571428571432,78.57142857142858,b)}[sigma=Scalar{exp=2.03}],ramp=Scalar{Cl(1e-10,0.5,b)}[sigma=Scalar{exp=2.03}],w_d=Log{Cl(-18.372511079725303,-12.372511079725303,b),

 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 0.3092138908562312
Updating fitness with value 1.419739717352189
Updating fitness with value 0.5733922369591159
Updating fitness with value 0.5885442680398176
Updating fitness with value 0.8360120629475899
Updating fitness with value 1.4015113963837118
Updating fitness with value 0.8470064015244456
Updating fitness with value 0.3976063716312779
Updating fitness with value 0.36075176938351716
Updating fitness with value 1.1589476434687378
Updating fitness with value 1.6225229623073938
Updating fitness with value 1.057564972860729
Updating fitness with value 0.13994424660992144
Updating fitness with value 1.0365296022823114
Updating fitness with value 0.3551479953388694
Updating fitness with value 1.5643618023956354
Updating fitness with value 1.4698353858150177
980 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 0.13994424660992144, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(2.75,68.75,b)}[sigma=Scalar{ex